In [1]:
from utils import *

## Creating Experiment in our local drive

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/isham993/Desktop/Programming-Tutorials/2023-Data-Science/mlflow-experiment-intermediate-level/mlruns/1', creation_time=1696819671257, experiment_id='1', last_update_time=1696819671257, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [4]:
len(df_train), len(df_val)

(73908, 61921)

Feature Engineering

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715204520257

In [10]:
os.makedirs('models', exist_ok=True)
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

Our first MLFlow run...

In [11]:
with mlflow.start_run():

    # for working in team
    mlflow.set_tag("developer", "isham")

    # Tracking the dataset
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.0025
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    # Tracking Hyper parameters
    mlflow.log_param("alpha", alpha)

    # Tracking Evaluation Metric
    mlflow.log_metric("rmse", rmse)

    # mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

## Part 2

In [12]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

def objective(params):
    with mlflow.start_run():
        # Setting up tags - developer and model
        mlflow.set_tag("model", "xgboost")
        mlflow.set_tag("developer", "isham")

        # Tracking hyperparams
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)

        # Tracking evaluation metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [13]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/isham993/mambaforge/envs/ml_experiment_mlflow/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [23:32:48] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.91070                          
[1]	validation-rmse:9.89101                           
[2]	validation-rmse:9.09692                           
[3]	validation-rmse:8.49075                           
[4]	validation-rmse:8.01947                           
[5]	validation-rmse:7.67707                           
[6]	validation-rmse:7.40897                           
[7]	validation-rmse:7.21032                           
[8]	validation-rmse:7.06475                           
[9]	validation-rmse:6.95301                           
[10]	validation-rmse:6.86271                          
[11]	validation-rmse:6.79801                          
[12]	validation-rmse:6.74803                          
[13]	validation-rmse:6.70805                          
[14]	validation-rmse:6.67607                          
[15]	validation-rmse:6.65268                          
[16]	validation-rmse:6.62911                          
[17]	validation-rmse:6.61485                          
[18]	valid

/Users/isham993/mambaforge/envs/ml_experiment_mlflow/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [23:33:44] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.30661                                                    
[1]	validation-rmse:9.02149                                                     
[2]	validation-rmse:8.17716                                                     
[3]	validation-rmse:7.63761                                                     
[4]	validation-rmse:7.29474                                                     
[5]	validation-rmse:7.07643                                                     
[6]	validation-rmse:6.93881                                                     
[7]	validation-rmse:6.84483                                                     
[8]	validation-rmse:6.78165                                                     
[9]	validation-rmse:6.74032                                                     
[10]	validation-rmse:6.70907                                                    
[11]	validation-rmse:6.68676                                                    
[12]	validation-rmse:6.66837

...

/Users/isham993/mambaforge/envs/ml_experiment_mlflow/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [23:39:30] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.44296                                                    
[1]	validation-rmse:7.97823                                                    
[2]	validation-rmse:7.25040                                                    
[3]	validation-rmse:6.89105                                                    
[4]	validation-rmse:6.71171                                                    
[5]	validation-rmse:6.61678                                                    
[6]	validation-rmse:6.56562                                                    
[7]	validation-rmse:6.53261                                                    
[8]	validation-rmse:6.51191                                                    
[9]	validation-rmse:6.49821                                                    
[10]	validation-rmse:6.48796                                                   
[11]	validation-rmse:6.48281                                                   
[12]	validation-rmse:6.47850            

/Users/isham993/mambaforge/envs/ml_experiment_mlflow/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [23:39:48] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.70333                                                   
[1]	validation-rmse:9.57721                                                    
[2]	validation-rmse:8.74527                                                    
[3]	validation-rmse:8.14574                                                    
[4]	validation-rmse:7.71184                                                    
[5]	validation-rmse:7.40713                                                    
[6]	validation-rmse:7.19083                                                    
[7]	validation-rmse:7.03124                                                    
[8]	validation-rmse:6.92027                                                    
[9]	validation-rmse:6.83614                                                    
[10]	validation-rmse:6.77584                                                   
[11]	validation-rmse:6.73261                                                   
[12]	validation-rmse:6.69797            

KeyboardInterrupt: 

Using best hyperparamters (with lowest RMSE)

In [15]:
best_params = {
    'learning_rate': 0.36207126540475176,
    'max_depth': 45,
    'min_child_weight': 1.8926075916196095,
    'objective': 'reg:linear',
    'reg_alpha': 0.3094232922808997,
    'reg_lambda': 0.05431420826786889,
    'seed': 42
}

In [17]:
mlflow.xgboost.autolog()

In [18]:
booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

2023/10/31 00:16:49 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'cfa55efc06eb443b926d46d18c93e8fb', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


/Users/isham993/mambaforge/envs/ml_experiment_mlflow/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [00:16:53] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.44296
[1]	validation-rmse:7.97823
[2]	validation-rmse:7.25040
[3]	validation-rmse:6.89105
[4]	validation-rmse:6.71171
[5]	validation-rmse:6.61678
[6]	validation-rmse:6.56562
[7]	validation-rmse:6.53261
[8]	validation-rmse:6.51191
[9]	validation-rmse:6.49821
[10]	validation-rmse:6.48796
[11]	validation-rmse:6.48281
[12]	validation-rmse:6.47850
[13]	validation-rmse:6.47502
[14]	validation-rmse:6.47145
[15]	validation-rmse:6.46534
[16]	validation-rmse:6.46273
[17]	validation-rmse:6.46028
[18]	validation-rmse:6.45724
[19]	validation-rmse:6.45366
[20]	validation-rmse:6.44941
[21]	validation-rmse:6.44677
[22]	validation-rmse:6.44281
[23]	validation-rmse:6.43901
[24]	validation-rmse:6.43565
[25]	validation-rmse:6.43250
[26]	validation-rmse:6.43018
[27]	validation-rmse:6.42761
[28]	validation-rmse:6.42511
[29]	validation-rmse:6.42244
[30]	validation-rmse:6.42019
[31]	validation-rmse:6.41848
[32]	validation-rmse:6.41625
[33]	validation-rmse:6.41291
[34]	validation-rmse:6.4

2023/10/31 00:17:33 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2023/10/31 00:17:33 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/isham993/mambaforge/envs/ml_experiment_mlflow/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [00:17:33] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
2023/10/31 00:17:34 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/isham993/mambaforge/envs/ml_experiment_mlflow/lib/python3.8/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. 